<a href="https://colab.research.google.com/github/nullpitch-dev/DS_L1_Notebooks/blob/master/Ex02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import math

url_pur = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/ds_purchase_log.csv'
url_cus = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/ds_customer_mst.csv'
url_pro = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/ds_product_mst.csv'

data_pur = pd.read_csv(url_pur)
data_cus = pd.read_csv(url_cus)
data_pro = pd.read_csv(url_pro)

In [2]:
# 회원별 연령대 정리
cus_group = data_cus.groupby(["cust_no", "update_dt"]).agg({"age_gender": "max"}).reset_index()
cus_group = cus_group.sort_values(by=["cust_no", "update_dt"], ascending=[True, False])
cus_group = cus_group.drop_duplicates(subset=["cust_no"], keep="first")

# 구매이력에 연령대 추가
base = pd.merge(data_pur, cus_group[["cust_no", "age_gender"]], how="left", left_on="cust_no", right_on="cust_no")

# 구매이력에 카테고리와 구매금액 추가
base = pd.merge(base, data_pro, how="left", on="item_no")

### [1]

In [3]:
# 연령과 성별을 분리
base = base.assign(age=base.apply(lambda x: x["age_gender"].split()[0], axis=1))
base = base.assign(gender=base.apply(lambda x: x["age_gender"].split()[1], axis=1))

In [4]:
# 성별과 대카테고리별 매출합과 회원수 계산
data1 = base.groupby(["gender", "item_ctg1"]).agg({"item_amt": "sum", "cust_no": "nunique"})
data1 = data1.assign(amt_per_cust=data1["item_amt"] / data1["cust_no"])  # 인당 매출액 계산
data1 = data1.sort_values(by=["gender", "amt_per_cust"], ascending=[False, False])  # 성별별 인당 매출액 역순으로 Sorting

In [5]:
mail = data1.loc["M"]["amt_per_cust"].index[1]
femail = data1.loc["F"]["amt_per_cust"].index[1]

In [6]:
print(f"{mail}, {femail}")

의류, 화장품


### [2]

In [7]:
# 회원별 매출액 계산
data2 = base.groupby("cust_no").agg({"item_amt": "sum"})
# Ranking
data2 = data2.assign(rank=data2.rank(ascending=False, method="min"))

In [8]:
# Class 정의
upper = data2.shape[0] * 0.2
lower = data2.shape[0] * 0.8
data2 = data2.assign(rankClass=data2["rank"].apply(lambda x: "top" if x <= upper else ("bottom" if x >= lower else "middle")))

In [9]:
# 매출 비율 계산
revenue = data2.groupby("rankClass").agg({"item_amt": "sum"})
top = revenue.loc["top"][0]
middle = revenue.loc["middle"][0]
bottom = revenue.loc["bottom"][0]

In [10]:
top_total = top / (top + middle + bottom)
top_bottom = top / bottom

print(f"{math.floor(top_total * 100)}, {math.floor(top_bottom)}")

49, 9


### [3]

In [11]:
data3 = base.groupby(["cust_no", "age"]).agg({"item_amt": "sum"}).reset_index()

In [12]:
data3 = data3.assign(age_cls=data3["age"].apply(lambda x: "G1" if x == "30대" else ("G2" if x == "40대" else "G3")))

In [13]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import MultiComparison

result = ols(formula="item_amt ~ C(age_cls)", data=data3).fit()
anova_table = anova_lm(result)
f_val = anova_table["F"][0]

print(f"{math.floor(f_val)}")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


11


In [14]:
# 사후검정
comparison = MultiComparison(data=data3["item_amt"], groups=data3["age_cls"])
print(comparison.tukeyhsd())

    Multiple Comparison of Means - Tukey HSD, FWER=0.05     
group1 group2  meandiff  p-adj    lower      upper    reject
------------------------------------------------------------
    G1     G2  8461.1735 0.001   4117.8207 12804.5262   True
    G1     G3   717.8831   0.9  -3454.2396  4890.0058  False
    G2     G3 -7743.2904 0.001 -12467.8902 -3018.6906   True
------------------------------------------------------------


* G2 - G1 = 8461 (G2의 평균 > G1의 평균)
* G3 - G1 = 717 (G3의 평균 > G1의 평균)
* G3 - G2 = -7743 (G2의 평균 > G3의 평균)
* ==> G2의 평균 > G3의 평균 > G1의 평균

* G1의 평균과 G2의 평균이 다르다고 할 수 없다 <== Reject
* G1의 평균과 G3의 평균이 다르다고 할 수 없다 <== Fail to reject
* G2의 평균과 G3의 평균이 다르다고 할 수 없다 <== Reject

### G2 >(다르다) G3 =(같다) G1

### [4]

In [96]:
# data4 = base.groupby("cust_no").agg({"item_ctg2": lambda x: list(set(x))})
# data4 = data4.assign(cnt=data4["item_ctg2"].apply(lambda x: len(x)))
# data4 = data4[data4["cnt"] >= 2]
# cat_list = list(data4["item_ctg2"])

# from mlxtend.preprocessing import TransactionEncoder
# from mlxtend.frequent_patterns import apriori, association_rules

# te = TransactionEncoder()
# te_array = te.fit(cat_list).transform(cat_list)
# te_df = pd.DataFrame(te_array, columns=te.columns_)
# freq_items = apriori(te_df, min_support=0.001, use_colnames=True)
# asso_rules = association_rules(freq_items, metric="confidence", min_threshold=0.001)
# asso_rules = asso_rules.assign(check=asso_rules.apply(lambda x: "농산물" in x["antecedents"], axis=1))
# asso_rules = asso_rules[asso_rules["check"]].sort_values(by="lift", ascending=False)

# print(f"{list(asso_rules.iloc[0, 1])[0]}")

가공식품


In [94]:
# Pivoting
pivot = base.pivot_table(index="cust_no", columns="item_ctg2", aggfunc="size", fill_value=0)
# 1 이상은 1, 0은 0으로
pivot = (pivot >= 1) + 0
# filtering 2건 이상 구매만
pivot = pivot.assign(cnt=pivot.apply(lambda x: x.sum(), axis=1))
pivot = pivot[pivot["cnt"] >= 2]
pivot = pivot.iloc[:, :-1] # cnt column 날리기

from mlxtend.frequent_patterns import apriori, association_rules

freq_items = apriori(pivot, min_support=0.001, use_colnames=True)
asso_rules = association_rules(freq_items, metric="confidence", min_threshold=0.001)

asso_rules = asso_rules.assign(check=asso_rules["antecedents"].apply(lambda x: "농산물" in x))
asso_rules = asso_rules[asso_rules["check"]].sort_values(by="lift", ascending=False)

print(f"{list(asso_rules.iloc[0, 1])[0]}")

가공식품
